In [51]:
import xml.etree.ElementTree as ET

# 입력 XML 파일 경로
xml_file = "box_toolpath.xml"

# 출력 CRS 파일 경로
output_file = "box_toolpath.crs"

# XML 파싱
tree = ET.parse(xml_file)
root = tree.getroot()

# 레이어별로 z 값 계산
points = []
layer_index = 0
start_z = 0.2
dz = 0.2

for layer in root.findall(".//Layer"):
    z_val = start_z + layer_index * dz
    for segment in layer.findall(".//Segment"):
        for point in segment.findall("Point"):
            x = float(point.attrib["x"])
            y = float(point.attrib["y"])
            z = z_val  # 고정된 z 오프셋
            points.append((x, y, z))
    layer_index += 1

# .crs 포맷으로 저장
with open(output_file, "w") as f:
    for i, (x, y, z) in enumerate(points):
        flag = i % 2
        f.write(f"{i}\t{x:.5f}\t{y:.5f}\t{z:.5f}\t{flag}\n")

print(f"✅ CRS file saved as '{output_file}'")


✅ CRS file saved as 'box_toolpath.crs'


In [52]:
import pandas as pd

# CRS 파일 경로
crs_file = "box_toolpath.crs"

# 파일 읽기 (탭으로 구분된 5열: timestep, x, y, z, flag)
df = pd.read_csv(crs_file, sep="\t", header=None, names=["timestep", "x", "y", "z", "flag"])

# 최소/최대값 계산
x_min, x_max = df["x"].min(), df["x"].max()
y_min, y_max = df["y"].min(), df["y"].max()
z_min, z_max = df["z"].min(), df["z"].max()

print(f"x: min={x_min}, max={x_max}")
print(f"y: min={y_min}, max={y_max}")
print(f"z: min={z_min}, max={z_max}")


x: min=131.79102, max=151.79297
y: min=123.74601, max=143.74791
z: min=0.4, max=4.2


In [53]:
import pandas as pd

# 파일 경로
input_file = "box_toolpath.crs"
output_file = input_file

# CRS 파일 읽기
df = pd.read_csv(input_file, sep="\t", header=None, names=["timestep", "x", "y", "z", "flag"])

# 현재 최소값
x_min, y_min = df["x"].min(), df["y"].min()

# 오프셋 계산
x_offset = -10 - x_min
y_offset = -10 - y_min

# 오프셋 적용
df["x"] += x_offset
df["y"] += y_offset

# 소수점 4자리까지 포맷 설정하여 저장
with open(output_file, "w") as f:
    for _, row in df.iterrows():
        f.write(f"{int(row['timestep'])}\t{row['x']:.4f}\t{row['y']:.4f}\t{row['z']:.4f}\t{int(row['flag'])}\n")

print(f"✅ Offset + formatting applied. Saved to '{output_file}'")


✅ Offset + formatting applied. Saved to 'box_toolpath.crs'


In [54]:
import pandas as pd
import numpy as np

# 파일 경로
input_file = "box_toolpath.crs"
output_file = input_file

# 유지할 Z 값 목록과 허용 오차
keep_z_list = [1.0, 2.0, 3.0, 4.0]
tol = 1e-4

# CRS 파일 읽기
df = pd.read_csv(input_file, sep="\t", header=None, names=["timestep", "x", "y", "z", "flag"])

# Z 값 필터링
filtered_df = pd.concat([
    df[np.abs(df["z"] - z_val) < tol] for z_val in keep_z_list
], ignore_index=True)

# 저장: 원래 형식 그대로 (탭 구분, 소수점 4자리)
with open(output_file, "w") as f:
    for _, row in filtered_df.iterrows():
        f.write(f"{int(row['timestep'])}\t{row['x']:.4f}\t{row['y']:.4f}\t{row['z']:.4f}\t{int(row['flag'])}\n")

print(f"✅ Z in {keep_z_list}만 필터링되어 저장됨 → '{output_file}'")

✅ Z in [1.0, 2.0, 3.0, 4.0]만 필터링되어 저장됨 → 'box_toolpath.crs'


In [55]:
import pandas as pd

# 파일 경로
input_file = "box_toolpath.crs"
output_file = input_file  # 덮어쓰기

# 파일 읽기
df = pd.read_csv(input_file, sep="\t", header=None, names=["timestep", "x", "y", "z", "flag"])

# ✅ timestep 열을 0부터 재번호
df["timestep"] = range(len(df))

# 저장: 탭 구분, 소수점 4자리 유지
with open(output_file, "w") as f:
    for _, row in df.iterrows():
        f.write(f"{int(row['timestep'])}\t{row['x']:.4f}\t{row['y']:.4f}\t{row['z']:.4f}\t{int(row['flag'])}\n")

print(f"✅ 1열 timestep을 0부터 재번호하여 저장 완료 → '{output_file}'")


✅ 1열 timestep을 0부터 재번호하여 저장 완료 → 'box_toolpath.crs'


In [56]:
import pandas as pd
import numpy as np

# 파일 경로
input_file = "box_toolpath.crs"
output_file = input_file  # 덮어쓰기

# 거리 대비 timestep 증가 비율
scale = 0.1

# 파일 읽기
df = pd.read_csv(input_file, sep="\t", header=None, names=["timestep", "x", "y", "z", "flag"])

# 거리 기반 timestep 계산
timestep = [0.0]  # 첫 번째는 항상 0.0
for i in range(1, len(df)):
    dx = df.loc[i, "x"] - df.loc[i-1, "x"]
    dy = df.loc[i, "y"] - df.loc[i-1, "y"]
    dz = df.loc[i, "z"] - df.loc[i-1, "z"]
    dist = np.sqrt(dx**2 + dy**2 + dz**2)
    timestep.append(timestep[-1] + scale * dist)

# 소수점 4자리까지 반영
df["timestep"] = [round(t, 4) for t in timestep]

# 저장: 탭 구분, 소수점 4자리 유지
with open(output_file, "w") as f:
    for _, row in df.iterrows():
        f.write(f"{row['timestep']:.4f}\t{row['x']:.4f}\t{row['y']:.4f}\t{row['z']:.4f}\t{int(row['flag'])}\n")

print(f"✅ 거리 기반 timestep을 소수점 4자리로 계산하여 저장 완료 → '{output_file}'")


✅ 거리 기반 timestep을 소수점 4자리로 계산하여 저장 완료 → 'box_toolpath.crs'
